In [ ]:
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 "numpy<2" --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install -q -U openmim "numpy<2"

In [ ]:
# Install MMCV compiled for PyTorch 2.0.1 (not 2.1.0)
!pip install -q mmcv-full==1.7.2 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html "numpy<2"

In [ ]:
!mim install -q mmdet\<3.0.0

In [ ]:
!pip uninstall -q mmrotate --yes
!rm -rf mmrotate
!git clone -q https://github.com/open-mmlab/mmrotate.git

In [ ]:
!cd mmrotate && pip install -q -e .

In [ ]:
!mkdir model
!wget -q -O ./model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth https://download.openmmlab.com/mmrotate/v0.1.0/redet/redet_re50_fpn_1x_dota_ms_rr_le90/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth

# When you reach this cell you MUST restart the session
# Do not re-run the cells before this one!

In [ ]:
import numpy as np
import torch
import mmcv
import cv2
from mmdet.apis    import init_detector
from mmrotate.apis import inference_detector_by_patches

In [ ]:
class Args:
    def __init__( self, batch_size: int, patch_size: int, patch_step: int ):
        self.img = "./input/1.png"
        self.config = "mmrotate/configs/redet/redet_re50_refpn_1x_dota_ms_rr_le90.py"
        self.checkpoint = "model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth"
        self.score_thr = 0.0025
        self.merge_iou_thr = 0.85
        self.img_ratios = [1.0]
        self.out_file = "PA-SM-2020-6-12-SM"
        self.batch_size = batch_size
        self.patch_sizes = [patch_size]
        self.patch_steps = [patch_step]
        self.device = 'cuda:0'
        self.palette = 'dota'

torch.cuda.set_per_process_memory_fraction( 0.85 )

### setup args here ###
args = Args( batch_size=24, patch_size=1024, patch_step=768 )
model = init_detector( args.config, args.checkpoint, device=args.device )

In [ ]:
import torch

def debug_model_devices(model):
    """Debug function to show device placement"""
    print("=== Model Device Debug ===")

    device_summary = {}

    for name, param in model.named_parameters():
        device = str(param.device)
        if device not in device_summary:
            device_summary[device] = []
        device_summary[device].append(name)

    for device, params in device_summary.items():
        print(f"Device {device}: {len(params)} parameters")
        if len(params) < 10:  # Show first few if not too many
            for param in params[:5]:
                print(f"  - {param}")

    return device_summary

# Use after model initialization
devices = debug_model_devices(model)

In [ ]:
img = mmcv.imread( args.img )

In [ ]:
result = inference_detector_by_patches( model,
                                        img,
                                        args.patch_sizes,
                                        args.patch_steps,
                                        args.img_ratios,
                                        args.merge_iou_thr,
                                        args.batch_size )

In [ ]:
model.show_result( args.img, result, out_file=args.out_file, score_thr=args.score_thr )